In [2]:
%iam_role arn:aws:iam::481237842548:role/CF-dev-TruveDevops-Glue-Role

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Current iam_role is None
iam_role has been set to arn:aws:iam::481237842548:role/CF-dev-TruveDevops-Glue-Role.


In [4]:
%region us-west-2

Previous region: None
Setting new region to: us-west-2
Region is set to: us-west-2


In [0]:
%additional_python_modules "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/sstm_transformation-0.1.0-py3-none-any.whl"
%number_of_workers 2

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

Additional python modules to be included:
s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/sstm_transformation-0.1.0-py3-none-any.whl
Previous number of workers: 5
Setting new number of workers to: 2
Authenticating with profile=default
glue_role_arn defined by user: arn:aws:iam::481237842548:role/CF-dev-TruveDevops-Glue-Role
Trying to create a Glue session for the kernel.
Worker Type: G.1X
Number of Workers: 2
Session ID: fdb1e73e-c69b-4c73-9c53-2551e7180934
Applying the following default arguments:
--glue_kernel_version 0.31
--enable-glue-datacatalog true
--additional-python-modules s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/sstm_transformation-0.1.0-py3-none-any.whl
Waiting for session fdb1e73e-c69b-4c73-9c53-2551e7180934 to get into ready status...
Session fdb1e73e-c69b-4c73-9c53-2551e7180934 has been created




In [1]:
#download configuration file
import boto3
session = boto3.Session()

s3 = session.resource('s3')
s3.Bucket('dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o').download_file('confs/filevine/sstm.yaml', 'sstm.yaml')

In [2]:
with open("sstm.yaml", "r") as f:
    sstm_config = f.read()

In [3]:
sstm_config

'org_id: 6586\ntsm:\n  - name: PeopleType\n    fields:\n    - name: Truve_Org_ID\n      data_type: int\n      transform: \n        source: internal\n        type: key\n    - name: Client_Org_ID\n      data_type: int\n      transform:\n        source: etl\n        type: org_id\n    - name: People_Type_ID\n      data_type: int\n    - name: People_Type\n      data_type: string\n      transform:\n        source: etl\n        source_entity_type: core\n        source_entity_name: contact\n        source_field: personTypes\n        type: data\n    - name: People_Sub_Type\n      data_type: string\n    - name: Custom1\n      data_type: string\n    - name: Custom2\n      data_type: string\n    - name: Custom3\n      data_type: string\n  - name: PeopleMaster\n    fields:\n    - name: Truve_Org_ID\n      data_type: int\n      transform: \n        source: internal\n        type: key\n    - name: Client_Org_ID\n      data_type: int\n      transform:\n        source: etl\n        type: org_id\n    - 

In [4]:
def readFromFile(ext, path) :
    fileContent = spark.read.format(ext).option("header", "true").option("mergeSchema", "true").load(path)
    return fileContent


In [5]:
def writeToFile(df_join , path, mode) :
    df_join.write.parquet(path,mode=mode)

In [6]:
spark.conf.set("spark.sql.parquet.enableVectorizedReader","false")
spark.conf.set("spark.sql.sources.partitionColumnTypeInference.enabled", "false")
spark.conf.set("spark.sql.parquet.writeLegacyFormat", "true")


In [1]:
spark.conf.set("spark.sql.parquet.enableVectorizedReader","false")
spark.conf.set("spark.sql.sources.partitionColumnTypeInference.enabled", "false")


In [ ]:
from sstm_transformation.tsm_builder import TSMBuilder

In [ ]:
#PeopleType Transformation
from awsglue.dynamicframe import DynamicFrame
builder = TSMBuilder("sstm.yaml", spark=spark)
def people_type_transformation():
    #Read Raw Data
    contact_dyf = glueContext.create_dynamic_frame_from_options(\
    connection_type = "s3", \
    connection_options = {
        "paths": ["s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/contact/historical_contacts.parquet"]}, \
    format = "parquet",
    )
    contact_sp_df = contact_dyf.toDF()
    sstm_peoplemaster_df = builder.build_peopletypes(contact_df=contact_sp_df)
    sstm_peoplemaster_dyf = DynamicFrame.fromDF(sstm_peoplemaster_df, glueContext, "sstm_peoplemaster")
    print(sstm_peoplemaster_dyf.count())
    sstm_peoplemaster_df.write.mode("overwrite").format("parquet").save("s3://dev-truve-devops-05-databr-bucketetlprocesseddata-h2m2xopoctot/peoplemaster/")
    print("PeopleTypes Processed successfully")

In [ ]:
def people_master_transformation():
    #Read Raw Data
    contact_dyf = glueContext.create_dynamic_frame_from_options(\
    connection_type = "s3", \
    connection_options = {
        "paths": ["s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/contact/historical_contacts.parquet"]}, \
    format = "parquet",
    )
    contact_sp_df = contact_dyf.toDF()
    print(contact_sp_df.count())
    sstm_peoplemaster_df = builder.build_peoplemaster(contact_df=contact_sp_df)
    trans_obj = DynamicFrame.fromDF(sstm_peoplemaster_df, glueContext, "sstm_peoplemaster")
    print(trans_obj)
    #sstm_peoplemaster_df.write.mode("overwrite").format("parquet").save("s3://dev-truve-devops-05-databr-bucketetlprocesseddata-h2m2xopoctot/peoplemaster/")
    #print("PeopleTypes Processed successfully")

In [ ]:
#people_type_transformation()
people_master_transformation()

In [ ]:
df_forms =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/18764/*/form/casesummary.parquet")


In [ ]:
df_forms.count()

In [ ]:
df_forms.printSchema()

In [ ]:
df_forms.show()

In [ ]:
from awsglue.dynamicframe import DynamicFrame

dyf_casesummary = DynamicFrame.fromDF(df_forms, glueContext, "nested")


In [ ]:
glueContext.write_dynamic_frame.from_options(frame = dyf_casesummary,
                                         connection_type = "s3",
                                         connection_options = {"path": "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/temp/casesummary/"},
                                         format = "parquet")

In [ ]:
writeToFile(df_forms,"s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/temp/casesummary/", "overwrite")


In [8]:
%stop_session

Stopping session: af9192d4-ce86-4bf6-b2ac-86a5c3e1cd2b
Stopped session.


In [27]:
df_forms =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/18764/*/collection/meds.parquet")


AnalysisException: 'Path does not exist: s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/18764/*/collection/meds.parquet;'


In [ ]:
df_forms.count()

In [ ]:
df_forms.printSchema()

In [ ]:
df_forms.show(n=10)

In [ ]:
writeToFile(df_forms,"s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/temp/meds/", "overwrite")


In [12]:
df_intake =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/18764/*/form/intake.parquet")


In [13]:
df_intake.count()

2902


In [14]:
df_intake.show(n=10)

+----------------------+--------------------+-------------------------+-------------------------------+------------------------+-----------------+---------------------+------------+--------------------+----------------------+-------------------------+--------------------+----------+------------------------+------------+--------+--------------------+------------+--------------------+--------------------+-------------------------------+---------------------------+----------------------------+--------------------+-------------------------------+---------------------------------+-------------------+-----------------------+---------------------------+---------------------------------+------------------------+------------------+-------------------+----------------+-------------------------------+--------------+---------------------------------+----------------------------------------+----------------+-------------+---------------+-----------------+-------------------------+------------+-----

In [11]:
df_intake.printSchema()

root
 |-- listallchildrenandages: string (nullable = true)
 |-- accidenttype: string (nullable = true)
 |-- timeandlocationofaccident: string (nullable = true)
 |-- wereauthoritiescalledtothescene: boolean (nullable = true)
 |-- ifyeswhopolicemanageretc: string (nullable = true)
 |-- sfpremisepictures: boolean (nullable = true)
 |-- descriptionofaccident: string (nullable = true)
 |-- policereport: boolean (nullable = true)
 |-- clientsautoinsurance: string (nullable = true)
 |-- damagetoclientsvehicle: string (nullable = true)
 |-- damagetodefendantsvehicle: string (nullable = true)
 |-- bywhomandwhere: string (nullable = true)
 |-- datetaken: date (nullable = true)
 |-- locationofclientsvehicle: string (nullable = true)
 |-- vehicletowed: boolean (nullable = true)
 |-- towedby: string (nullable = true)
 |-- towedto: string (nullable = true)
 |-- estimatedone: boolean (nullable = true)
 |-- listcarriers: string (nullable = true)
 |-- amount: string (nullable = true)
 |-- injuriessusta

In [ ]:
writeToFile(df_intake,"s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/temp/intake/", "overwrite")


In [17]:
df_casesummary =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/18764/*/form/casesummary.parquet")


In [18]:
df_casesummary.count()

440


In [19]:
df_casesummary.show()

+-----+--------------------+---------+--------------+---------+---------------+--------------+---------+-----------------------+-----------------------+----------------------+--------------+---------------+-------------------+----------+--------------------+-------+----------------+---------------------+--------------------+----+--------------+----------------+----------------------+-------------------------+----------------------+--------------+-----------------+-------------------+-----------------+--------------------------+------------------+------------------------+--------------+----------------------+-----------------------+--------------+--------------+---------------------+----------------+-----------------+------------------------+-----+---------+-----------------+-------------------------+--------------------------+------------+---------------------+----------------------+----------------+---------+-------------+----------------+---------------------------+------------------

In [7]:
def write_parquet_files(df, name):
    df_rept = df.repartition(1)
    writeToFile(df_rept,"s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/temp/{}/".format(name), "overwrite")


In [9]:
df_intake =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/18764/*/form/intake.parquet")


In [10]:
write_parquet_files(df_intake, "intake")

In [11]:
df_intake.count()

2902


In [12]:
df_project =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/18764/*/project.parquet")
write_parquet_files(df_project, "project")

In [13]:
df_meds =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/18764/*/collections/meds.parquet")
write_parquet_files(df_meds, "meds")